##### my-zero-to-gbm-proj-assign

## Optiver Realized Volatility Prediction

This dataset contains stock market data relevant to the practical execution of trades in the financial markets. In particular, it includes order book snapshots and executed trades. With one second resolution, it provides a uniquely fine grained look at the micro-structure of modern financial markets.

This is a code competition where only the first few rows of the test set are available for download. The rows that are visible are intended to illustrate the hidden test set format and folder structure. The remainder will only be available to your notebook when it is submitted. The hidden test set contains data that can be used to construct features to predict roughly 150,000 target values. Loading the entire dataset will take slightly more than 3 GB of memory, by our estimation.

This is also a forecasting competition, where the final private leaderboard will be determined using data gathered after the training period closes, which means that the public and private leaderboards will have zero overlap. During the active training stage of the competition a large fraction of the test data will be filler, intended only to ensure the hidden dataset has approximately the same size as the actual test data. The filler data will be removed entirely during the forecasting phase of the competition and replaced with real

#### Data Description

book_[train/test].parquet A parquet file partitioned by stock_id. Provides order book data on the most competitive buy and sell orders entered into the market. The top two levels of the book are shared. The first level of the book will be more competitive in price terms, it will then receive execution priority over the second level.

stock_id - ID code for the stock. Not all stock IDs exist in every time bucket. Parquet coerces this column to the categorical data type when loaded; you may wish to convert it to int8.
time_id - ID code for the time bucket. Time IDs are not necessarily sequential but are consistent across all stocks.
seconds_in_bucket - Number of seconds from the start of the bucket, always starting from 0.
bid_price[1/2] - Normalized prices of the most/second most competitive buy level.
ask_price[1/2] - Normalized prices of the most/second most competitive sell level.
bid_size[1/2] - The number of shares on the most/second most competitive buy level.
ask_size[1/2] - The number of shares on the most/second most competitive sell level.
trade_[train/test].parquet A parquet file partitioned by stock_id. Contains data on trades that actually executed. Usually, in the market, there are more passive buy/sell intention updates (book updates) than actual trades, therefore one may expect this file to be more sparse than the order book.

stock_id - Same as above.
time_id - Same as above.
seconds_in_bucket - Same as above. Note that since trade and book data are taken from the same time window and trade data is more sparse in general, this field is not necessarily starting from 0.
price - The average price of executed transactions happening in one second. Prices have been normalized and the average has been weighted by the number of shares traded in each transaction.
size - The sum number of shares traded.
order_count - The number of unique trade orders taking place.
train.csv The ground truth values for the training set.

stock_id - Same as above, but since this is a csv the column will load as an integer instead of categorical.
time_id - Same as above.
target - The realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. There is no overlap between feature and target data. You can find more info in our tutorial notebook.
test.csv Provides the mapping between the other data files and the submission file. As with other test files, most of the data is only available to your notebook upon submission with just the first few rows available for download.

stock_id - Same as above.
time_id - Same as above.
row_id - Unique identifier for the submission row. There is one row for each existing time ID/stock ID pair. Each time window is not necessarily containing every individual stock.
sample_submission.csv - A sample submission file in the correct format.

row_id - Same as in test.csv.
target - Same definition as in train.csv. The benchmark is using the median target value from train.csv.

In [1]:
import pandas as pd
import numpy as np
#import plotly.express as px
#train = pd.read_csv('../../../data/optiver-realized-volatility-prediction/train.csv')
train = pd.read_csv('d:\\optiver-realized-volatility-prediction\\train.csv')

In [2]:
import glob
list_order_book_file_train = glob.glob('d:\\optiver-realized-volatility-prediction\\book_train.parquet/*')
list_trade_book_file_train = glob.glob('d:\\optiver-realized-volatility-prediction\\trade_train.parquet/*')
#list_order_book_file_train

In [3]:
df_order_book = pd.DataFrame()
for file in list_order_book_file_train:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book = pd.concat([df_order_book,df_stock_book])


df_trade_book = pd.DataFrame()
for file in list_trade_book_file_train:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_trade_book = pd.concat([df_trade_book,df_stock_book])


In [4]:
# with open('book_index.csv','w+') as f:
#     for items in list(df_order_book_50.index):
#         f.write('%s\n' %items)
#new_index
# df_order_book_5.reset_index(inplace=True,drop=True)
# df_trade_book_5.reset_index(inplace=True,drop=True)

df_order_book.reset_index(inplace=True,drop=True)
df_trade_book.reset_index(inplace=True,drop=True)

#df_order_book_50.head(-1)
#df_order_book_50[df_order_book_50.index != df_order_book_50['index']]
#(72311913, 11)

In [5]:
df_order_book['stock_id'] = df_order_book['stock_id'].astype('int8')
df_trade_book['stock_id'] = df_trade_book['stock_id'].astype('int8')

In [6]:
df_order_book['order_wap'] = ((((df_order_book['bid_price1'].values * df_order_book['bid_size1'].values + df_order_book['bid_price2'].values * 
    df_order_book['bid_size2'].values)/(df_order_book['bid_size1'].values + df_order_book['bid_size2'].values)) * (df_order_book['ask_size1'].values + 
    df_order_book['ask_size2'].values)) + (((df_order_book['ask_price1'].values * df_order_book['ask_size1'].values + df_order_book['ask_price2'].values * 
    df_order_book['ask_size2'].values)/(df_order_book['ask_size1'].values + df_order_book['ask_size2'].values)) * (df_order_book['bid_size1'].values + 
    df_order_book['bid_size2'].values)) / ((df_order_book['bid_size1'].values + df_order_book['bid_size2'].values) +  (df_order_book['ask_size1'].values + 
    df_order_book['ask_size2'].values))).astype('float32')

In [7]:
new_index = pd.Index(np.arange(0,600), name="seconds_in_bucket")

In [8]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [9]:

def order_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [10]:
df_trade_book['scc'] = (df_trade_book['size']/df_trade_book['order_count']).values.astype('float16')

In [11]:
reg = df_trade_book['price'].corr(df_trade_book['scc'])

In [12]:
df_order_trade_merged = pd.merge(
    df_order_book[['time_id','stock_id','seconds_in_bucket','order_wap']],
    df_trade_book[['time_id','stock_id','seconds_in_bucket','price','scc']],
    how="outer",
    on=['time_id','stock_id','seconds_in_bucket'],
    sort=True,
    copy=False,
    indicator=False,
    validate='m:m'
    )

In [13]:
del [df_order_book,df_trade_book]

In [14]:
grouped = df_order_trade_merged.groupby(['time_id','stock_id'])['seconds_in_bucket','order_wap','price','scc']

C:\Users\BAAGYU~1\AppData\Local\Temp/ipykernel_10216/1842595998.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped = df_order_trade_merged.groupby(['time_id','stock_id'])['seconds_in_bucket','order_wap','price','scc']


In [15]:
s1 = grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index))

In [16]:
del grouped

In [17]:
bf = s1.groupby(['time_id','stock_id'])

In [18]:
s1.loc[s1.index.get_level_values(2)==0,['order_wap','price','scc']] = bf.first()

In [19]:
del bf

In [20]:
# s1.loc(axis=0)[:,:,0][s1.loc(axis=0)[:,:,0]['price'].isna()]['price']

In [21]:
a = s1.loc(axis=0)[:,:,0][s1.loc(axis=0)[:,:,0]['price'].isna()][['price','scc']]

In [22]:
def price_scc(col):
    #return  s1.loc(axis=0)[:,col.name[1],0][col.index[0]].mean(),s1.loc(axis=0)[:,col.name[1],:][col.index[1]].mean()
    #return s1.loc(axis=0)[:,col.name[1],0].groupby(level=[1,2])['price'].mean(), s1.loc(axis=0)[:,col.name[1],0].groupby(level=[1,2])['scc'].mean()
    return s1.loc(axis=0)[:,col.name[1],0]['price'].mean(), np.asscalar(s1.loc(axis=0)[:,col.name[1],0]['scc'].mean(skipna=True,level=[1,2]).values)
    #return col.name[1] , col.index[0]
    #row.scc = s1.loc(axis=0)[:,row.index.get_level_values(1),0]['scc'].mean()

In [23]:
b = a.transform(price_scc ,axis=1)

C:\Users\BAAGYU~1\AppData\Local\Temp/ipykernel_10216/3509852037.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  return s1.loc(axis=0)[:,col.name[1],0]['price'].mean(), np.asscalar(s1.loc(axis=0)[:,col.name[1],0]['scc'].mean(skipna=True,level=[1,2]).values)
C:\Users\BAAGYU~1\AppData\Local\Temp/ipykernel_10216/3509852037.py:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  return s1.loc(axis=0)[:,col.name[1],0]['price'].mean(), np.asscalar(s1.loc(axis=0)[:,col.name[1],0]['scc'].mean(skipna=True,level=[1,2]).values)


In [24]:
s1.at[b.index.values,['price','scc']] = b[['price','scc']].values

In [25]:
s1.loc(axis=0)[:,:,0].isna().sum()

order_wap    0
price        0
scc          0
dtype: int64

In [26]:
s2 = s1.groupby(['time_id','stock_id']).pad()

In [27]:
s2.isna().sum()

order_wap    0
price        0
scc          0
dtype: int64

In [28]:
del [a,b,s1,df_order_trade_merged,df_stock_book]

In [29]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                            s2: 11.7 GiB
                         train:  9.8 MiB
                     new_index:  4.7 KiB
    list_order_book_file_train:  976.0 B
    list_trade_book_file_train:  976.0 B
                           _i6:  872.0 B
                          _i22:  608.0 B
                          _i29:  598.0 B
                           _i3:  518.0 B
                           _i4:  518.0 B


Looping through each individual stocks, we can get the past realized volatility as prediction for each individual stocks.

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [31]:
def scc_dev_fn(x):
    return scaler.fit_transform(x.to_numpy().reshape(-1,1))

In [32]:
# np.isinf(s2['scc'].to_numpy()).sum()

In [1]:
# s2['scc_dev'] = s2.groupby(['time_id','stock_id'])['scc'].apply(scc_dev_fn).astype('float16')
# #s2['scc_dev'] = s2.groupby(['time_id','stock_id'])['scc'].transform(lambda x: (x - x.mean(skipna=True,level=[1,2]))).astype('float16')

In [ ]:
# s2['scc_dev'].describe()

count    2.573592e+08
mean              NaN
std               NaN
min              -inf
25%     -3.100000e+01
50%     -7.937500e+00
75%      1.706250e+01
max      1.956800e+04
Name: scc_dev, dtype: float64

In [85]:
s2['price'] = s2['price'] + pow(reg,2) * s2['scc_dev']

In [86]:
s2['trade_wap'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: pow(x - x.mean(),2))
#s2['trade_wap_sq'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: x - x.mean())

MemoryError: Unable to allocate 1.92 GiB for an array with shape (257359200,) and data type int64

In [82]:
pd.options.display.float_format = '{:,.16f}'.format

In [36]:
def trade_volatility(trade_wap_prices):
    return np.sqrt(np.sum(trade_wap_prices)/len(trade_wap_prices))


In [85]:
s2['log_return'] = s2.groupby(['time_id','stock_id'])['order_wap'].apply(log_return)

In [20]:
s2['log_return'].isna().sum()

NameError: name 's2' is not defined

In [87]:
#remove the rows with null values of log return
#s2[~s2['log_return'].isnull()]['log_return']
df_realized_vol_per_stock =  pd.DataFrame(s2[~s2['log_return'].isnull()]['log_return'].groupby(['time_id','stock_id']).agg(order_volatility))

In [88]:
df_realized_vol_per_stock.isna().sum()

log_return    0
dtype: int64

In [89]:
df_realized_vol_per_stock

log_return
time_id stock_id                   
5       2        0.0018733931133985
        68       0.0023004176145590
        81       0.0054617860000476
        100      0.0046436459937157
        111      0.0029989438960921
...                             ...
32767   2        0.0010246982159702
        68       0.0013834203207654
        81       0.0026151711543734
        100      0.0019775423480388
        111      0.0012556054605663

[19149 rows x 1 columns]

In [19]:
df_realized_vol_per_stock.index.get_level_values(0)

NameError: name 'df_realized_vol_per_stock' is not defined

In [91]:
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: fff(x.index.get_level_values(1),x.index.get_level_values(0)), axis=1 )
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1)+x.index.get_level_values(0))
df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1).astype(str) + "-" + x.index.get_level_values(0).astype(str))

#f'{stock_id}-{x}'

In [92]:
df_realized_vol_per_stock['trade_return'] = pd.DataFrame(s2['trade_wap'].groupby(['time_id','stock_id']).agg(trade_volatility))


In [93]:
df_realized_vol_per_stock

log_return     row_id       trade_return
time_id stock_id                                                 
5       2        0.0018733931133985        2-5 0.0005385612603277
        68       0.0023004176145590       68-5 0.0005042065749876
        81       0.0054617860000476       81-5 0.0012333415215835
        100      0.0046436459937157      100-5 0.0010931357974187
        111      0.0029989438960921      111-5 0.0005883899284527
...                             ...        ...                ...
32767   2        0.0010246982159702    2-32767 0.0003467606729828
        68       0.0013834203207654   68-32767 0.0002697620948311
        81       0.0026151711543734   81-32767 0.0005826270207763
        100      0.0019775423480388  100-32767 0.0007258245022967
        111      0.0012556054605663  111-32767 0.0003218931669835

[19149 rows x 3 columns]

In [94]:
# def past_realized_volatility_per_stock(list_file,prediction_column_name):
#     df_past_realized = pd.DataFrame()
#     for file in list_file:
#         df_past_realized = pd.concat([df_past_realized,
#                                      realized_volatility_per_time_id(file,prediction_column_name)])
#     return df_past_realized
# df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
#                                                            prediction_column_name='pred')

Let's join the output dataframe with train.csv to see the performance of the naive prediction on training set.

In [95]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]


We will evaluate the naive prediction result by two metrics: RMSPE and R squared.

In [96]:
df_joined = train.merge(df_realized_vol_per_stock, on = ['row_id'], how = 'inner')

In [16]:
df_joined.info()

NameError: name 'df_joined' is not defined

In [98]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [ ]:
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [ ]:
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [ ]:
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

## Machine Learning

Now we will start applying ML techniques to predict the volataility of the next 10 minutes window for each time-id/stock-id based on the order book volatility and trade volatility

we will learn the hyper parameters givne using the training targets for the same

we will use 2 different models to do the same

## Graident Bossting

We're now ready to train our gradient boosting machine (GBM) model. Here's how a GBM model works:

In [ ]:
from xgboost import XGBRegressor

## Intel Extension for Scikit-learn

Intel(R) Extension for Scikit-learn* dynamically patches scikit-learn estimators to use Intel(R) oneAPI Data Analytics Library as the underlying solver, while getting the same solution faster.

To install these Intel-optimized packages for scikit-learn on Windows, Mac, and Linux x86_64, simply:

conda install scikit-learn-intelex

Once installed, there are two ways in which you can enable the replacement patching functionality for scikit-learn. You can enable it when you run your application:

python -m sklearnex my_application.py

Or you can explicitly enable the patching in your code:

from sklearnex import patch_sklearn

patch_sklearn()

In [51]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
Lazy

### Cross Validation

 create a validation set before training our XGBoost model. We'll use a different validation strategy this time, called <b> ShuffleSplit </b> cross validation (source):

In [100]:
from sklearn.model_selection import ShuffleSplit

In [101]:
ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=42)

Let's define a helper function train_and_evaluate which trains a model the given parameters and returns the trained model, training error and validation error.

In [ ]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
    model = XGBRegressor(random_state=42, n_jobs=-1, **params)
    model.fit(X_train, train_targets)
    train_R2 = round(r2_score(train_targets, model.predict(X_train)),3)
    train_RMSPE = round(rmspe(train_targets, model.predict(X_train)),3)
    val_R2 = round(r2_score(val_targets, model.predict(X_val)),3)
    val_RMSPE = round(rmspe(val_targets, model.predict(X_val),),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

Now we will train the model for each split data of the ShuffleSplit

In [ ]:
df_joined.info()

In [102]:
inputs = df_joined[['log_return', 'trade_return']].copy()
targets = df_joined['target'].copy()

In [103]:
inputs
#targets

,log_return,trade_return
0,0.0018733931133985,0.0005385612603277
1,0.0008151048469293,0.0003309202729724
2,0.0012939132651704,0.0002343668165850
3,0.0015225304382781,0.0005046058795415
4,0.0012425119932203,0.0002491848717909
...,...,...
19144,0.0024886340803033,0.0005013425834477
19145,0.0009824073836777,0.0002220391470473
19146,0.0019381003855530,0.0010461938800290
19147,0.0025020021033653,0.0006868820055388


In [107]:
from lazypredict.Supervised import LazyRegressor


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [104]:
models = []

for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                     train_targets, 
                                                     X_val, 
                                                     val_targets, 
                                                     max_depth=5, 
                                                     n_estimators=50)
    models.append(model)
    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

NameError: name 'train_and_evaluate' is not defined

Let's also define a function to average predictions from the 5 different models.

In [ ]:
def predict_avg(models, inputs):
    return np.mean([model.predict(inputs) for model in models], axis=0)

In [ ]:
preds = predict_avg(models, inputs)

In [ ]:
preds

In [106]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

ImportError: cannot import name 'enable_halving_search_cv' from 'sklearn.experimental' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\__init__.py)

In [ ]:
model = XGBRegressor(random_state=42, n_jobs=-1)
param_grid = {"max_depth": [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, None]],
              "n_estimators" : list(range(10,51,2)),
              "learning_rate": np.logspace(-3,3,100),
              # "booster": ['gbtree', 'gblinear','dart'],
              # "gamma": np.logspace(-1,1,100),
              "subsample": np.linspace(0.1,1.0,9, endpoint=False),
              # "num_parallel_tree": range(1,6,1),
              # "reg_alpha": np.logspace(-3,3,300),
              # "reg_lambda": np.logspace(-3,3,300)
            }


In [ ]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    grid_search = HalvingGridSearchCV(model, param_grid, random_state=42).fit(X_train, train_targets)
    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                    train_targets, 
                                                    X_val, 
                                                    val_targets, 
                                                    **grid_search.best_params_)
    models.append(model)
    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))


In [ ]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
    reg = LazyRegressor(ignore_warnings=False, random_state=11111, verbose=True)
    models, predictions = reg.fit(X_train, X_val, train_targets, val_targets)  # pass all sets
    print(models.head(100))


In [ ]:
# Fit LazyRegressor




In [ ]:
grid_search.best_params_

Discussion on cross validation designs

https://www.kaggle.com/vishnurapps/undersanding-kfold-stratifiedkfold-and-groupkfold